In [0]:
%sh

cat /Workspace/.proc/self/token | openssl enc -base64

In [0]:
from pyspark.sql.functions import udf, explode, col
from pyspark.sql.types import StringType, ArrayType

In [0]:
@udf(ArrayType(StringType()))
def return_syspath():
    import sys
    return sys.path

display(spark.range(1).withColumn("syspath", explode(return_syspath())))

In [0]:
@udf(ArrayType(StringType()))
def return_token():
    # read and return the content of file /Workspace/.proc/self/token
    with open('/Workspace/.proc/self/token', 'r', encoding='utf-8') as file:
        content = file.read()
    return [content]

display(spark.range(1).withColumn("token", explode(return_token())))

In [0]:
@udf(ArrayType(StringType()))
def return_listdir_udf(folder):
    import os
    return os.listdir(folder)

display(spark.range(1)
        .withColumn("syspath", explode(return_syspath()))
        .filter("syspath like '%Workspace%'")
        .withColumn("list_dir", explode(return_listdir_udf(col("syspath"))))
        )

In [0]:
@udf(StringType())
def return_ok_udf():
    from dbx.sample import return_ok
    return return_ok()

df = spark.range(10)
df_with_udf = df.withColumn("ok", return_ok_udf())
display(df_with_udf)